<a href="https://colab.research.google.com/github/nastya-biran/scheduler/blob/main/%D0%A1%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 79.7 MB/s eta 0:00:00


In [2]:
from pulp import *
import numpy as np
import pandas as pd
import re

In [20]:
teachers_data = pd.read_csv("Teachers.csv")
groups_data = pd.read_csv("Groups.csv")
limits_data = pd.read_csv("Teachers_limits.csv")

In [21]:
teachers_data.head(5)

,Преподаватель,"Зельеварение, семинар","Зельеварение, лекция","Защита от темных сил, лекция","Защита от темных сил, семинар","Трансфигурация, лекция","Трансфигурация, семинар",Понедельник,Вторник,Среда,Четверг,Пятница,Суббота
0,Северус Снейп,очно,очно,0,очно,0,0,NaN,"16.20-17.40, 18.10-19.30",NaN,"9.30-10.50, 11.10-12.30",NaN,NaN
1,Минерва Макгоналл,0,0,очно,онлайн,0,очно,"11.10-12.30, 13.00-14.20",NaN,"14.40-16.00, 16.20-17.40",NaN,NaN,NaN
2,Альбус Дамблдор,онлайн,0,0,0,онлайн,онлайн,"11.10-12.30, 13.00-14.20",NaN,NaN,"9.30-10.50, 11.10-12.30",NaN,NaN


In [22]:
groups_data.head(5)

,Номер группы,"Зельеварение, лекция","Зельеварение, семинар","Защита от темных сил, лекция","Защита от темных сил, семинар","Трансфигурация, лекция","Трансфигурация, семинар",Майнор,Военная кафедра
0,1,2,2,1,1,0,0,1,1
1,2,2,2,1,1,0,0,0,1
2,3,0,0,1,1,2,2,1,0
3,4,0,0,1,1,2,2,0,0


In [23]:
limits_data.head(5)

,Преподаватель,Все пары в один день,Дни без пар,Количество пар в неделю
0,Северус Снейп,NaN,"Понедельник, Суббота",5-6
1,Минерва Макгоналл,Да,NaN,4-5
2,Альбус Дамблдор,NaN,NaN,5-6


In [24]:
all_slots = ['9.30-10.50', '11.10-12.30', '13.00-14.20', '14.40-16.00', '16.20-17.40', '18.10-19.30', '19.40-21.00']
all_days = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота']
all_subjects = list(groups_data.columns.values)[1:-2]
all_teachers = list(pd.unique(teachers_data["Преподаватель"]))
all_groups = list(pd.unique(groups_data["Номер группы"]))
print(all_subjects)
print(all_teachers)
print(all_groups)

['Зельеварение, лекция', 'Зельеварение, семинар', 'Защита от темных сил, лекция', 'Защита от темных сил, семинар', 'Трансфигурация, лекция', 'Трансфигурация, семинар']
['Северус Снейп', 'Минерва Макгоналл', 'Альбус Дамблдор']
[1, 2, 3, 4]


In [25]:
def get_subjects_by_teacher(teacher):
  teacher_data = teachers_data[teachers_data["Преподаватель"] == teacher]
  teacher_data = teacher_data.filter(all_subjects)
  available_subjects = list(teacher_data.columns[teacher_data.isin(["очно", "онлайн"]).any()].values)
  subject_inds = [all_subjects.index(subject) for subject in available_subjects]
  return subject_inds

def get_groups_by_subject(subject):
  subject_data = groups_data[groups_data[subject] >= 1]
  available_groups = list(pd.unique(subject_data["Номер группы"]))
  group_inds = [all_groups.index(group) for group in available_groups]
  return group_inds

def get_answers(teacher):
  teacher_data = teachers_data[teachers_data["Преподаватель"] == teacher]
  answers = []
  for day in all_days:
    if pd.isna(teacher_data.iloc[0][day]):
      answers.append(None)
    else:
      slot_inds = [all_slots.index(slot) for slot in teacher_data.iloc[0][day].split(", ")]
      answers.append(slot_inds)
  return answers



def get_subjects_by_group(group):
  group_data = groups_data[groups_data["Номер группы"] == group]
  group_data = group_data.filter(all_subjects)
  needed_subjects = list(group_data.where(group_data >= 1).columns.values)
  subject_inds = [all_subjects.index(subject) for subject in needed_subjects]
  return subject_inds


def get_teachers_by_subject(subject):
  subject_data = teachers_data[teachers_data[subject].isin(["очно", "онлайн"])]
  available_teachers = list(pd.unique(subject_data["Преподаватель"]))
  teacher_inds = [all_teachers.index(teacher) for teacher in available_teachers]
  return teacher_inds

def get_classes_per_week_for_teacher(teacher):
  teacher_limits_data = limits_data[limits_data["Преподаватель"] == teacher].iloc[0]
  if pd.notna(teacher_limits_data["Количество пар в неделю"]):
    classes_nums = teacher_limits_data["Количество пар в неделю"].split("-")
  return int(classes_nums[0]), int(classes_nums[1])

def get_format(teacher, subject):
  teacher_data = teachers_data[teachers_data["Преподаватель"] == teacher].iloc[0]
  return teacher_data[subject]

def check_for_minor_and_military_day(group):
  group_data = groups_data[groups_data["Номер группы"] == group].iloc[0]
  has_minor = group_data["Майнор"]
  has_military_day = group_data["Военная кафедра"]
  return has_minor, has_military_day


In [26]:
pref_table = np.zeros((len(groups_data), len(all_subjects), len(teachers_data), len(all_days), len(all_slots)))

for k, teacher in enumerate(all_teachers):
  available_subjects = get_subjects_by_teacher(teacher)
  available_slots = get_answers(teacher)
  for j in available_subjects:
    available_groups = get_groups_by_subject(all_subjects[j])
    for i in available_groups:
      for l, slots_for_day in enumerate(available_slots):
        if slots_for_day is not None:
          for m in slots_for_day:
            pref_table[i, j, k, l, m] = 1


In [27]:
M = 1e5

prob = LpProblem("schedule", LpMaximize)

attends = LpVariable.dicts("x", ((i, j, k, l, m)
                                  for i in range(len(all_groups))
                                  for j in range(len(all_subjects))
                                  for k in range(len(all_teachers))
                                  for l in range(len(all_days))
                                  for m in range(len(all_slots))),
                            cat="Binary")


#Условие 2.1
for i in range(len(all_groups)):
  needed_subjects = get_subjects_by_group(all_groups[i])
  #print(f"Группа {all_groups[i]}")
  for j in needed_subjects:
    #Группа должна посещать указанное количество лекций/семинаров каждого нужного предмета у кого-то из учителей, которые ведут этот предмет
    available_teachers = get_teachers_by_subject(all_subjects[j])
    num = groups_data[groups_data["Номер группы"] == all_groups[i]][all_subjects[j]].iloc[0]
    #print(all_subjects[j], num)
    prob += lpSum([attends[(i, j, k, l, m)]
                  for k in available_teachers
                  for l in range(len(all_days))
                  for m in range(len(all_slots))]) == num
    #Нельзя ходить на лекцию/семинар к учителю, который не ведет этот предмет
    unavailable_teachers = list(set(range(len(all_teachers))) - set(available_teachers))
    prob += lpSum([attends[(i, j, k, l, m)]
                  for k in unavailable_teachers
                  for l in range(len(all_days))
                  for m in range(len(all_slots))]) == 0
  #Нельзя ходить на ненужные предметы
  unneeded_subjects = list(set(range(len(all_subjects))) - set(needed_subjects))
  for j in unneeded_subjects:
    prob += lpSum([attends[(i, j, k, l, m)]
                  for k in range(len(all_teachers))
                  for l in range(len(all_days))
                  for m in range(len(all_slots))]) == 0




#Условие 2.2
#Группа посещает не более одного слота одновременно
for i in range(len(all_groups)):
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      prob += lpSum([attends[(i, j, k, l, m)]
                     for j in range(len(all_subjects))
                     for k in range(len(all_teachers))]) <= 1

#Условие 2.3
#Между онлайн и очной парой как минимум 3 слота
h = LpVariable.dicts("h", ((i, l, m)
                    for i in range(len(all_groups))
                    for l in range(len(all_days))
                    for m in range(len(all_slots))),
               cat="Binary")
for i in range(len(all_groups)):
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      prob +=  lpSum([attends[(i, j, k, l, m)] * (get_format(all_teachers[k], all_subjects[j]) == "онлайн")
                for j in range(len(all_subjects))
                for k in range(len(all_teachers))]) == h[(i, l, m)]

t = LpVariable.dicts("t", ((i, l, m)
                    for i in range(len(all_groups))
                    for l in range(len(all_days))
                    for m in range(len(all_slots))),
               cat="Binary")

for i in range(len(all_groups)):
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      prob += lpSum([attends[(i, j, k, l, m)] * (get_format(all_teachers[k], all_subjects[j]) == "очно")
                for j in range(len(all_subjects))
                for k in range(len(all_teachers))]) == t[(i, l, m)]
'''for i in range(len(all_groups)):
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      prob += 7* h[(i, l, m)] + lpSum([t[(i, l, m1)] 
                                        for m1 in range(max(0, m - 3), min(len(all_slots), m + 3))])<= 7'''


#Условие 2.4
#В день майнора и военки основных пар не должно быть
for i in range(len(all_groups)):
  has_minor, has_military_day = check_for_minor_and_military_day(all_groups[i])
  if has_minor == 1:
    prob += lpSum([attends[(i, j, k, 2, m)] 
                     for j in range(len(all_subjects))
                     for k in range(len(all_teachers))
                     for m in range(len(all_slots))]) == 0
  if has_military_day == 1:
    prob += lpSum([attends[(i, j, k, 3, m)] 
                     for j in range(len(all_subjects))
                     for k in range(len(all_teachers))
                     for m in range(len(all_slots))]) == 0


#Условие 3.1
#Преподаватель ведет семинар не более чем у одной группы одновременно, лекцию не более чем у количества групп, которые записаны
for k in range(len(all_teachers)):
  for j in range(len(all_subjects)):
    num_classes = 1;
    if "лекция" in all_subjects[j]:
      num_classes = len(get_groups_by_subject(all_subjects[j]))
    for l in range(len(all_days)):
      for m in range(len(all_slots)):
        prob += lpSum([attends[(i, j, k, l, m)]
                      for i in range(len(all_groups))]) <= num_classes


#Условие 3.2 
#Каждый преподаватель ведет не более одного предмета одновременно

u = LpVariable.dicts("u", ((j, k, l, m)
                    for j in range(len(all_subjects))
                    for k in range(len(all_teachers))
                    for l in range(len(all_days))
                    for m in range(len(all_slots))),
               cat="Binary")


for j in range(len(all_subjects)):
  for k in range(len(all_teachers)):
    for l in range(len(all_days)):
      for m in range(len(all_slots)):
        prob += M * u[(j, k, l, m)] >= lpSum([attends[(i, j, k, l, m)]
                                        for i in range(len(all_groups))])
    

for k in range(len(all_teachers)):
  for l in range(len(all_days)):
      for m in range(len(all_slots)):
          prob += lpSum([u[(j, k, l, m)] for j in range(len(all_subjects))]) <= 1


#Условие 3.6
#Если это лекция, то на ней должны присутствовать все записанные группы одновременно

for j in range(len(all_subjects)):
  if "лекция" in all_subjects[j]:
    groups_num = len(get_groups_by_subject(all_subjects[j]))
    for k in range(len(all_teachers)):
      for l in range(len(all_days)):
        for m in range(len(all_slots)):
          prob += u[(j, k, l, m)] * groups_num == lpSum([attends[(i, j, k, l, m)] for i in range(len(all_groups))])

#Условие 3.7 
#Лекции/семинары должен вести один и тот же преподаватель

q = LpVariable.dicts("q", ((i, j, k)
                    for i in range(len(all_groups))
                    for j in range(len(all_subjects))
                    for k in range(len(all_teachers))),
               cat="Binary")


for i in range(len(all_groups)):
  for j in range(len(all_subjects)):
    for k in range(len(all_teachers)):
      prob += M * q[(i, j, k)] >= lpSum([attends[(i, j, k, l, m)]
                                        for l in range(len(all_days))
                                        for m in range(len(all_slots))])
      
for i in range(len(all_groups)):
  for j in range(len(all_subjects)):
    prob += lpSum([q[(i, j, k)] for k in range(len(all_teachers))])  <= 1



#Условие 3.3
#Отсутствие пар в определенные дни
for k in range(len(all_teachers)):
  forbidden_days_answer = limits_data[limits_data["Преподаватель"] == all_teachers[k]].iloc[0]["Дни без пар"]
  if pd.notna(forbidden_days_answer):
    forbidden_days = [all_days.index(day) for day in forbidden_days_answer.split(", ")]
    print(forbidden_days)
    for l in forbidden_days:
      prob += lpSum([attends[(i, j, k, l, m)]
                    for i in range(len(all_groups))
                    for j in range(len(all_subjects))
                    for m in range(len(all_slots))]) == 0

#Условие 3.4
#Все пары в один день
y = LpVariable.dicts("y", ((k, l)
                    for k in range(len(all_teachers))
                    for l in range(len(all_days))),
               cat="Binary")

for k in range(len(all_teachers)):
  for l in range(len(all_days)):
    prob += M * y[(k, l)] >= lpSum([attends[(i, j, k, l, m)]
                                    for i in range(len(all_groups))
                                    for j in range(len(all_subjects))
                                    for m in range(len(all_slots))])
    

for k in range(len(all_teachers)):
  in_one_day_answer = limits_data[limits_data["Преподаватель"] == all_teachers[k]]
  if in_one_day_answer.iloc[0]["Все пары в один день"] == "Да": 
    prob += lpSum([y[(k, l)] for l in range(len(all_days))]) == 1

#Условие 3.5
#Ограничение по количеству пар для преподавателей

for k in range(len(all_teachers)):
  mn, mx = get_classes_per_week_for_teacher(all_teachers[k])
  prob += mn <= lpSum([u[(j, k, l, m)]
                 for j in range(len(all_subjects))
                 for l in range(len(all_days))
                 for m in range(len(all_slots))]) <= mx


#Целевые функция
#Совпадения с предпочтениями преподавателей
tgt1 = LpVariable("tgt1", 
                  cat="Integer")

prob += lpSum([pref_table[i, j, k, l, m] * attends[(i, j, k, l, m)]
               for i in range(len(all_groups))
               for j in range(len(all_subjects))
               for k in range(len(all_teachers)) 
               for l in range(len(all_days)) 
               for m in range(len(all_slots))]) == tgt1
#prob += 0.5 * tgt1

#Минимизация окон
w1 = LpVariable.dicts("w1", ((i, l, m)
                    for i in range(len(all_groups))
                    for l in range(len(all_days))
                    for m in range(len(all_slots))),
               cat="Binary")
w2 = LpVariable.dicts("w2", ((i, l, m)
                    for i in range(len(all_groups))
                    for l in range(len(all_days))
                    for m in range(len(all_slots))),
               cat="Binary")
for i in range(len(all_groups)):
  for l in range(len(all_days)):
    prob += lpSum([w1[(i, l, m1)]
                for m1 in range(len(all_slots))]) <= 1
    prob += lpSum([w2[(i, l, m1)]
                for m1 in range(len(all_slots))]) <= 1
    for m in range(len(all_slots)):
      prob += w1[(i, l, m)] <= h[(i, l, m)] + t[(i, l, m)]
      prob += lpSum([w1[(i, l, m1)]
                for m1 in range(m + 1)]) >= h[(i, l, m)] + t[(i, l, m)]      
      prob += w2[(i, l, m)] <= h[(i, l, m)] + t[(i, l, m)]
      prob += lpSum([w2[(i, l, m1)]
                for m1 in range(m, len(all_slots))]) >= h[(i, l, m)] + t[(i, l, m)]

tgt2 = LpVariable("tgt2", 
                  cat="Integer")

windows = LpVariable.dicts("windows", ((i, l)
                    for i in range(len(all_groups))
                    for l in range(len(all_days))),
               cat="Integer")

classes_per_day = LpVariable.dicts("cl_day", ((i, l)
                    for i in range(len(all_groups))
                    for l in range(len(all_days))),
               cat="Integer")

for i in range(len(all_groups)):
    for l in range(len(all_days)):
      prob += lpSum([h[(i, l, m)] + t[(i, l, m)]
                        for m in range(len(all_slots))]) == classes_per_day[(i, l)]

has_classes = LpVariable.dicts("has_cl", ((i, l)
                    for i in range(len(all_groups))
                    for l in range(len(all_days))),
               cat="Binary")

for i in range(len(all_groups)):
    for l in range(len(all_days)):
      prob += M * has_classes[(i, l)] >= classes_per_day[(i, l)]
      prob += has_classes[(i, l)] <= classes_per_day[(i, l)]

for i in range(len(all_groups)):
    for l in range(len(all_days)):
      prob += lpSum([w2[(i, l, m)] * m
                      for m in range(len(all_slots))])- lpSum([w1[(i, l, m)] * m
                                                        for m in range(len(all_slots))]) - classes_per_day[(i, l)] + has_classes[(i, l)] == windows[(i, l)]


prob += lpSum([windows[(i, l)]
              for i in range(len(all_groups))
              for l in range(len(all_days))]) == tgt2

#Минимизация количества учебных дней
tgt3 = LpVariable("tgt3", 
                  cat="Integer")

prob += lpSum([has_classes[(i, l)] 
              for i in range(len(all_groups))
              for l in range(len(all_days))]) == tgt3

#Минимизация количества дней, в которых есть хотя бы одна онлайн пара

online_classes_per_day = LpVariable.dicts("on_cl_day", ((i, l)
                    for i in range(len(all_groups))
                    for l in range(len(all_days))),
               cat="Integer")

for i in range(len(all_groups)):
    for l in range(len(all_days)):
      prob += lpSum([h[(i, l, m)]
                        for m in range(len(all_slots))]) == online_classes_per_day[(i, l)]

has_online_classes = LpVariable.dicts("has_on_cl", ((i, l)
                    for i in range(len(all_groups))
                    for l in range(len(all_days))),
               cat="Binary")

for i in range(len(all_groups)):
    for l in range(len(all_days)):
      prob += M * has_online_classes[(i, l)] >= online_classes_per_day[(i, l)]
      prob += has_online_classes[(i, l)] <= online_classes_per_day[(i, l)]

tgt4 = LpVariable("tgt4", 
                  cat="Integer")

prob += lpSum([has_online_classes[(i, l)] 
              for i in range(len(all_groups))
              for l in range(len(all_days))]) == tgt4

prob += 0.6 * tgt1 - 0.2 * tgt2 - 0.15 * tgt3 - 0.05 * tgt4

[0, 5]


In [28]:
time_limit_in_seconds = 60

prob.solve(PULP_CBC_CMD(msg=1, timeLimit=time_limit_in_seconds))

1

In [29]:
print("Значение целевой функции:", prob.objective.value())

Значение целевой функции: 6.95


In [30]:
prob_res = prob.variablesDict()
res = np.chararray((len(all_teachers), len(all_slots), len(all_days)), itemsize=100, unicode=True).astype(str)
res[:] = ""
for k in range(len(all_teachers)):
  print(f"\nПреподаватель:{all_teachers[k]}")
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      subjects = set()
      for j in range(len(all_subjects)):
        for i in range(len(all_groups)):
          if prob_res[f"x_({i},_{j},_{k},_{l},_{m})"].value() == 1:
            res[k, m, l] += str(all_groups[i]) + ','
            subjects.add(all_subjects[j])
      if (len(res[k, m, l]) > 0):
        res[k, m, l] += str(list(subjects))
        format = get_format(all_teachers[k], str(list(subjects)[0]))
        res[k, m, l] += ", " + format
        print(f"{all_days[l]} {all_slots[m]}  {res[k, m, l]}")

pd.DataFrame(res[0], columns=all_days, index=all_slots)


Преподаватель:Северус Снейп
Вторник 13.00-14.20  1,['Зельеварение, семинар'], очно
Вторник 14.40-16.00  1,['Зельеварение, семинар'], очно
Вторник 16.20-17.40  1,2,['Зельеварение, лекция'], очно
Вторник 18.10-19.30  1,2,['Зельеварение, лекция'], очно
Вторник 19.40-21.00  2,['Защита от темных сил, семинар'], очно
Четверг 9.30-10.50  4,['Защита от темных сил, семинар'], очно

Преподаватель:Минерва Макгоналл
Понедельник 9.30-10.50  3,['Защита от темных сил, семинар'], онлайн
Понедельник 11.10-12.30  1,2,3,4,['Защита от темных сил, лекция'], очно
Понедельник 13.00-14.20  1,['Защита от темных сил, семинар'], онлайн
Понедельник 16.20-17.40  4,['Трансфигурация, семинар'], очно
Понедельник 18.10-19.30  4,['Трансфигурация, семинар'], очно

Преподаватель:Альбус Дамблдор
Понедельник 13.00-14.20  3,4,['Трансфигурация, лекция'], онлайн
Понедельник 14.40-16.00  3,4,['Трансфигурация, лекция'], онлайн
Вторник 13.00-14.20  2,['Зельеварение, семинар'], онлайн
Вторник 14.40-16.00  2,['Зельеварение, семин

,Понедельник,Вторник,Среда,Четверг,Пятница,Суббота
9.30-10.50,,,,"4,['Защита от темных сил, семинар'], очно",,
11.10-12.30,,,,,,
13.00-14.20,,"1,['Зельеварение, семинар'], очно",,,,
14.40-16.00,,"1,['Зельеварение, семинар'], очно",,,,
16.20-17.40,,"1,2,['Зельеварение, лекция'], очно",,,,
18.10-19.30,,"1,2,['Зельеварение, лекция'], очно",,,,
19.40-21.00,,"2,['Защита от темных сил, семинар'], очно",,,,


In [31]:
res_groups = np.chararray((len(all_groups), len(all_slots), len(all_days)), itemsize=100, unicode=True).astype(str)
res_groups[:] = ""
for i in range(len(all_groups)):
  print(f"\nГруппа:{all_groups[i]}")
  for l in range(len(all_days)):
    for m in range(len(all_slots)):
      for k in range(len(all_teachers)):
        for j in range(len(all_subjects)):
            if prob_res[f"x_({i},_{j},_{k},_{l},_{m})"].value() == 1:
              format = get_format(all_teachers[k], all_subjects[j])
              res_groups[i, m, l] += all_subjects[j] +", " + all_teachers[k] + ", " + format
      if (len(res_groups[i, m, l]) > 0):
        print(f"{all_days[l]} {all_slots[m]}  {res_groups[i, m, l]}")

pd.DataFrame(res_groups[3], columns=all_days, index=all_slots)


Группа:1
Понедельник 11.10-12.30  Защита от темных сил, лекция, Минерва Макгоналл, очно
Понедельник 13.00-14.20  Защита от темных сил, семинар, Минерва Макгоналл, онлайн
Вторник 13.00-14.20  Зельеварение, семинар, Северус Снейп, очно
Вторник 14.40-16.00  Зельеварение, семинар, Северус Снейп, очно
Вторник 16.20-17.40  Зельеварение, лекция, Северус Снейп, очно
Вторник 18.10-19.30  Зельеварение, лекция, Северус Снейп, очно

Группа:2
Понедельник 11.10-12.30  Защита от темных сил, лекция, Минерва Макгоналл, очно
Вторник 13.00-14.20  Зельеварение, семинар, Альбус Дамблдор, онлайн
Вторник 14.40-16.00  Зельеварение, семинар, Альбус Дамблдор, онлайн
Вторник 16.20-17.40  Зельеварение, лекция, Северус Снейп, очно
Вторник 18.10-19.30  Зельеварение, лекция, Северус Снейп, очно
Вторник 19.40-21.00  Защита от темных сил, семинар, Северус Снейп, очно

Группа:3
Понедельник 9.30-10.50  Защита от темных сил, семинар, Минерва Макгоналл, онлайн
Понедельник 11.10-12.30  Защита от темных сил, лекция, Минерв

,Понедельник,Вторник,Среда,Четверг,Пятница,Суббота
9.30-10.50,,,,"Защита от темных сил, семинар, Северус Снейп, ...",,
11.10-12.30,"Защита от темных сил, лекция, Минерва Макгонал...",,,,,
13.00-14.20,"Трансфигурация, лекция, Альбус Дамблдор, онлайн",,,,,
14.40-16.00,"Трансфигурация, лекция, Альбус Дамблдор, онлайн",,,,,
16.20-17.40,"Трансфигурация, семинар, Минерва Макгоналл, очно",,,,,
18.10-19.30,"Трансфигурация, семинар, Минерва Макгоналл, очно",,,,,
19.40-21.00,,,,,,
